In [1]:
import json
import tensorflow as tf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from tensorflow import keras
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [1]:
from gensim.models import KeyedVectors
from cogworks_data.language import get_data_path

def load_glove(*,limit=None, filename = "glove.6B.200d.txt.w2v"):
    # this takes a while to load -- keep this in mind when designing your capstone project
    glove = KeyedVectors.load_word2vec_format(get_data_path(filename), binary=False, limit=limit)
    return glove

In [3]:
file_path1 = r"C:\Users\ejian\CogWorks-2022-Gausslien-Final-Capstone\Sarcasm Detector\Sarcasm_Headlines_Dataset.json"
df = pd.read_json(file_path1,lines=True)
df = df[['headline','is_sarcastic']]
df

,headline,is_sarcastic
0,former versace store clerk sues over secret 'b...,0
1,the 'roseanne' revival catches up to our thorn...,0
2,mom starting to fear son's web series closest ...,1
3,"boehner just wants wife to listen, not come up...",1
4,j.k. rowling wishes snape happy birthday in th...,0
...,...,...
26704,american politics in moral free-fall,0
26705,america's best 20 hikes,0
26706,reparations and obama,0
26707,israeli ban targeting boycott supporters raise...,0
